In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

import warnings
from copy import deepcopy
import gc
from collections import Counter

%matplotlib inline

In [31]:
from tqdm import tqdm

In [2]:
%cd ../

In [21]:
cat_features = ["type", "priority", "is_privatecategory", "class", "is_in_yandex", "mailctg", "directctg", "mailtype"]

def to_cat_features(df):
    for cat_feature in cat_features:
        df[cat_feature] = df[cat_feature].apply(lambda text: str(text))
    return df
        
df = to_cat_features(pd.read_csv('data/train_prepared.csv'))
test_df = to_cat_features(pd.read_csv('data/test_df_prepared.csv'))
y = pd.read_csv('data/train_y_prepared.csv')

In [24]:
test_ids = pd.read_csv('data/test_ids.csv')

In [22]:
def score(pred, y_true):
    tn, fp, fn, tp = confusion_matrix(y_true, pred).ravel()
    tpr = (tp) / (tp + fn)
    fpr = (fp) / (fp + tn)
    r_score = (tp) / (tp + fn)
    ra_score = (1 + tpr - fpr) / 2
    score = 0.1 * r_score + 0.9 * ra_score
    return score
    

def predict(models, X, threshold=0.5):
    preds = []
    for model in tqdm(models):
        preds.append(model.predict_proba(X)[:, 1])
    preds = np.array(preds) 
    preds = preds.mean(0) > threshold
    return preds

In [32]:
models = []

In [33]:
#!c1.32
for state in range(0, 10):
        X_train, X_val, y_train, y_val = train_test_split(df, y, test_size=0.2, random_state=state)
        model = CatBoostClassifier(random_state=state, max_depth=8, verbose=200, iterations=2000, auto_class_weights="Balanced", cat_features=cat_features)
        # print(X_train.head(), y_train.head())
        model.fit(X_train, y_train) 
        models.append(model)
        print(f'MODEL {state} score: {score(predict([model], X_val), y_val)}')

Learning rate set to 0.203601
0:	learn: 0.5058721	total: 1.02s	remaining: 34m 1s
200:	learn: 0.2626192	total: 2m 58s	remaining: 26m 40s
400:	learn: 0.2557737	total: 5m 57s	remaining: 23m 47s
600:	learn: 0.2508552	total: 8m 55s	remaining: 20m 45s
800:	learn: 0.2461723	total: 11m 58s	remaining: 17m 56s
1000:	learn: 0.2425140	total: 14m 56s	remaining: 14m 54s
1200:	learn: 0.2385799	total: 18m 1s	remaining: 11m 59s
1400:	learn: 0.2352684	total: 21m 4s	remaining: 9m
1600:	learn: 0.2320549	total: 24m 6s	remaining: 6m
1800:	learn: 0.2291993	total: 27m 11s	remaining: 3m
1999:	learn: 0.2267698	total: 30m 14s	remaining: 0us


100%|██████████| 1/1 [00:04<00:00,  4.50s/it]


MODEL 0 score: 0.8800285685116038
Learning rate set to 0.203601
0:	learn: 0.5060251	total: 1.09s	remaining: 36m 17s
200:	learn: 0.2634472	total: 2m 57s	remaining: 26m 30s
400:	learn: 0.2563013	total: 5m 59s	remaining: 23m 54s
600:	learn: 0.2508113	total: 9m 3s	remaining: 21m 4s
800:	learn: 0.2461862	total: 12m 5s	remaining: 18m 6s
1000:	learn: 0.2423104	total: 15m 8s	remaining: 15m 6s
1200:	learn: 0.2389621	total: 18m 10s	remaining: 12m 5s
1400:	learn: 0.2356340	total: 21m 12s	remaining: 9m 3s
1600:	learn: 0.2324825	total: 24m 12s	remaining: 6m 2s
1800:	learn: 0.2295577	total: 27m 15s	remaining: 3m
1999:	learn: 0.2268689	total: 30m 18s	remaining: 0us


100%|██████████| 1/1 [00:05<00:00,  5.02s/it]


MODEL 1 score: 0.8821934647840843
Learning rate set to 0.203601
0:	learn: 0.5156549	total: 1.05s	remaining: 34m 55s
200:	learn: 0.2629709	total: 2m 58s	remaining: 26m 39s
400:	learn: 0.2561551	total: 6m 3s	remaining: 24m 8s
600:	learn: 0.2507522	total: 9m 7s	remaining: 21m 14s
800:	learn: 0.2465767	total: 12m 10s	remaining: 18m 13s
1000:	learn: 0.2421846	total: 15m 13s	remaining: 15m 11s
1200:	learn: 0.2386007	total: 18m 13s	remaining: 12m 7s
1400:	learn: 0.2352184	total: 21m 16s	remaining: 9m 5s
1600:	learn: 0.2321830	total: 24m 18s	remaining: 6m 3s
1800:	learn: 0.2294298	total: 27m 21s	remaining: 3m 1s
1999:	learn: 0.2267854	total: 30m 24s	remaining: 0us


...

MODEL 7 score: 0.8783892167440385
Learning rate set to 0.203601
0:	learn: 0.5048246	total: 1.02s	remaining: 33m 52s
200:	learn: 0.2627404	total: 2m 59s	remaining: 26m 46s
400:	learn: 0.2557730	total: 6m 2s	remaining: 24m 5s
600:	learn: 0.2506454	total: 9m 5s	remaining: 21m 9s
800:	learn: 0.2460070	total: 12m 9s	remaining: 18m 12s
1000:	learn: 0.2418513	total: 15m 15s	remaining: 15m 13s
1200:	learn: 0.2385438	total: 18m 19s	remaining: 12m 11s
1400:	learn: 0.2352600	total: 21m 26s	remaining: 9m 9s
1600:	learn: 0.2320909	total: 24m 29s	remaining: 6m 6s
1800:	learn: 0.2292739	total: 27m 32s	remaining: 3m 2s
1999:	learn: 0.2265579	total: 30m 32s	remaining: 0us


100%|██████████| 1/1 [00:04<00:00,  4.81s/it]


MODEL 8 score: 0.8786705070941863
Learning rate set to 0.203601
0:	learn: 0.5175608	total: 998ms	remaining: 33m 14s
200:	learn: 0.2626942	total: 3m	remaining: 26m 54s
400:	learn: 0.2556921	total: 6m 3s	remaining: 24m 10s
600:	learn: 0.2506298	total: 9m 8s	remaining: 21m 16s
800:	learn: 0.2462032	total: 12m 12s	remaining: 18m 16s
1000:	learn: 0.2421990	total: 15m 17s	remaining: 15m 15s
1200:	learn: 0.2384633	total: 18m 22s	remaining: 12m 13s
1400:	learn: 0.2351628	total: 21m 24s	remaining: 9m 9s
1600:	learn: 0.2320707	total: 24m 29s	remaining: 6m 6s
1800:	learn: 0.2292808	total: 27m 33s	remaining: 3m 2s
1999:	learn: 0.2267517	total: 30m 38s	remaining: 0us


100%|██████████| 1/1 [00:04<00:00,  4.99s/it]


MODEL 9 score: 0.8790399312798443


In [34]:
test_preds = predict(models, test_df, 0.41).astype(int)

100%|██████████| 10/10 [04:54<00:00, 29.45s/it]


In [35]:
submission = pd.DataFrame({'id': test_ids.id, 'label': test_preds})

In [36]:
submission.to_csv("submission.csv", index=False)

In [37]:
test_preds.mean()

0.18442375